# **Heart Disease Data Analysis**

**Introduction:**
\
\
&emsp; &emsp; Presently, cardiovascular disease (CVD) has become one of the leading causes of death globally, and mortality rates due to CVD have been rising over the past three decades. To illustrate, in 1990, there were 12.1 million deaths caused by CVD globally, and an increase to 18.6 million deaths in 2019. CVD includes any condition which may impact the cardiovascular system, such as coronary heart disease, heart failure, stroke and more. Due to the large quantity of cardiovascular conditions which impact patients globally and have subsequently led to a significant mortality rate, it is important to develop a diagnostic system which can indicate whether or not one has CVD as early as possible to ensure immediate treatment and possible recovery. This leads to our project’s central question: Based on a patient’s resting systolic blood pressure, serum cholesterol levels and maximum achieved heart rate, do they have cardiovascular disease (CVD)? 
\
\
&emsp; &emsp;The purpose of our project is to create a classifying system which can use 3 predictors, resting systolic blood pressure, serum cholesterol levels and maximum achieved heart rate to determine whether or not a patient has CVD. The classification system will be made using a heart disease dataset obtained from UC Irvine’s Machine Learning Repository which contains data collected by the Cleveland Clinic Foundation. The dataset has 302 data points and a total of 14 attributes, 4 of which are continuous, 9 of which are discrete and 1 predicted attribute specifying whether or not an individual has heart disease. As mentioned, 3 continuous variables have been chosen as predictors for our classifier. 


**Preliminary Exploratory Data Analysis:**

In [364]:
import pandas as pd
import altair as alt
import numpy as np
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

The dataset, directly read from the web:

In [365]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
hd_original_data = pd.read_csv(url, header=None)
hd_original_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


To clean and wrangle this dataset into a tidy format, column names are added in the order given by the original website. The last column, initially referred to as "num", has been renamed to "heart disease presence", and refers to the heart disease status of the patient. This status ranges from 0-4, 0 indicating no presence of heart disease. The original column "thalach" has also been renamed to "max heart rate", and refers to the maximum heart rate achieved. All missing values have been dropped.

In [395]:
hd_original_data.columns = ["age", "sex", "cp", "trestbps(systolic)", "chol", "fbs", "restecg", "max_heart_rate", "exang", "oldpeak", "slope", "ca", "thal", "heart_disease_presence"]
hd_original_data['heart_disease_presence'] = pd.Categorical(hd_original_data.heart_disease_presence)
hd_data = hd_original_data[(hd_original_data['age'] != "?")
                           & (hd_original_data['sex'] != "?")
                           & (hd_original_data['trestbps(systolic)'] != "?")
                           & (hd_original_data['chol'] != "?")
                           & (hd_original_data['fbs'] != "?")
                           & (hd_original_data['restecg'] != "?")
                           & (hd_original_data['max_heart_rate'] != "?")
                           & (hd_original_data['exang'] != "?")
                           & (hd_original_data['oldpeak'] != "?")
                           & (hd_original_data['slope'] != "?")
                           & (hd_original_data['ca'] != "?")
                           & (hd_original_data['thal'] != "?")
                           & (hd_original_data['heart_disease_presence'] != "?")]
hd_data


,age,sex,cp,trestbps(systolic),chol,fbs,restecg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease_presence
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57.0,0.0,4.0,140.0,241.0,0.0,0.0,123.0,1.0,0.2,2.0,0.0,7.0,1
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3


For our project, we will split 75% of the data to use as our training data and the other 25% as our testing data.

In [397]:
hd_train, hd_test = train_test_split(hd_data, test_size=0.25, random_state=123) 
hd_train

,age,sex,cp,trestbps(systolic),chol,fbs,restecg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease_presence
278,57.0,1.0,2.0,154.0,232.0,0.0,2.0,164.0,0.0,0.0,1.0,1.0,3.0,1
259,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,1
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,0
186,42.0,1.0,3.0,120.0,240.0,1.0,0.0,194.0,0.0,0.8,3.0,0.0,7.0,0
172,59.0,0.0,4.0,174.0,249.0,0.0,0.0,143.0,1.0,0.0,2.0,0.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,57.0,1.0,3.0,128.0,229.0,0.0,2.0,150.0,0.0,0.4,2.0,1.0,7.0,1
83,68.0,1.0,3.0,180.0,274.0,1.0,2.0,150.0,1.0,1.6,2.0,0.0,7.0,3
17,54.0,1.0,4.0,140.0,239.0,0.0,0.0,160.0,0.0,1.2,1.0,0.0,3.0,0
233,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,0


To explore our dataset, we found the count and percentage of each level of heart disease presence:

In [379]:

explore_hd_grouped = (hd_train.groupby('heart_disease_presence').count())
explore_hd = explore_hd_grouped[["age"]].rename(columns={"age":"count"})
explore_hd = explore_hd.assign(
    percentage=100*explore_hd['count']/len(hd_train)
)
explore_hd

,count,percentage
heart_disease_presence,,
0,120,54.054054
1,43,19.369369
2,24,10.810811
3,25,11.261261
4,10,4.504505


The means of each predicator for the individual heart disease presence:

In [380]:
hd_predictors = hd_train[["trestbps(systolic)", "chol", "max_heart_rate", "heart_disease_presence"]]
hd_predictors.columns = ["trestbps(systolic) mean", "chol mean", "max_heart_rate mean", "heart_disease_presence"]
hd_0 = hd_predictors[hd_predictors["heart_disease_presence"] == 0]
hd_0 = pd.DataFrame(hd_0.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_0

hd_1 = hd_predictors[hd_predictors["heart_disease_presence"] == 1]
hd_1 = pd.DataFrame(hd_1.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_1.index=['1']

hd_2 = hd_predictors[hd_predictors["heart_disease_presence"] == 2]
hd_2 = pd.DataFrame(hd_2.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_2.index=['2']

hd_3 = hd_predictors[hd_predictors["heart_disease_presence"] == 3]
hd_3 = pd.DataFrame(hd_3.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_3.index=['3']

hd_4 = hd_predictors[hd_predictors["heart_disease_presence"] == 4]
hd_4 = pd.DataFrame(hd_4.drop(["heart_disease_presence"], axis=1).apply(np.mean)).transpose()
hd_4.index=['4']

hd_all = [hd_0, hd_1, hd_2, hd_3, hd_4]

hd_mean2 = pd.concat(hd_all)
hd_mean2.index.name = "heart_disease_presence"
hd_mean2

,trestbps(systolic) mean,chol mean,max_heart_rate mean
heart_disease_presence,,,
0,129.800000,239.250000,159.191667
1,134.186047,248.744186,146.441860
2,136.750000,263.041667,131.625000
3,136.360000,250.600000,130.440000
4,137.600000,263.300000,142.600000


Graphs representing these means:

In [393]:
hd_mean2_ = hd_mean2.reset_index()
hd_mean2_["heart_disease_presence"] = pd.Categorical(hd_mean2_.heart_disease_presence)
hdp_vs_max_htrt = (
    alt.Chart(hd_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("max_heart_rate mean", title="Maximum heart rate (BPM)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_max_htrt


/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [392]:
hdp_vs_chol = (
    alt.Chart(hd_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("chol mean", title="Serum cholesterol level (mg/dl)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_chol

alt.Chart(...)

In [391]:
hdp_vs_restbps = (
    alt.Chart(hd_mean2_)
    .mark_bar()
    .encode(
        x=alt.X("heart_disease_presence", title="Heart disease presence"),
        y=alt.Y("trestbps(systolic) mean", title="Systolic resting blood pressure (mm Hg)"),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence", scale=alt.Scale(scheme='dark2'))
    )
).configure_axis(titleFontSize=12)
hdp_vs_restbps

alt.Chart(...)

Since our research question focuses more on the general question of whether we can predict a person has heart disease or not, we have a modified table representing 0 as no and 1-4 as yes.

In [383]:
hd_train["heart_disease_presence"] = (hd_train['heart_disease_presence']).astype(str)
hd_general = hd_train.replace({"heart_disease_presence": {"0":"no", "1":"yes", "2":"yes", "3":"yes", "4":"yes"}})
hd_general



,age,sex,cp,trestbps(systolic),chol,fbs,restecg,max_heart_rate,exang,oldpeak,slope,ca,thal,heart_disease_presence
278,57.0,1.0,2.0,154.0,232.0,0.0,2.0,164.0,0.0,0.0,1.0,1.0,3.0,yes
259,57.0,1.0,2.0,124.0,261.0,0.0,0.0,141.0,0.0,0.3,1.0,0.0,7.0,yes
7,57.0,0.0,4.0,120.0,354.0,0.0,0.0,163.0,1.0,0.6,1.0,0.0,3.0,no
186,42.0,1.0,3.0,120.0,240.0,1.0,0.0,194.0,0.0,0.8,3.0,0.0,7.0,no
172,59.0,0.0,4.0,174.0,249.0,0.0,0.0,143.0,1.0,0.0,2.0,0.0,3.0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,57.0,1.0,3.0,128.0,229.0,0.0,2.0,150.0,0.0,0.4,2.0,1.0,7.0,yes
83,68.0,1.0,3.0,180.0,274.0,1.0,2.0,150.0,1.0,1.6,2.0,0.0,7.0,yes
17,54.0,1.0,4.0,140.0,239.0,0.0,0.0,160.0,0.0,1.2,1.0,0.0,3.0,no
233,74.0,0.0,2.0,120.0,269.0,0.0,2.0,121.0,1.0,0.2,1.0,1.0,3.0,no


In [388]:
chol_vs_restbps = (
    alt.Chart(hd_general)
    .mark_circle()
    .encode(
        x=alt.X("chol", title="Serum cholesterol level (mg/dl)", scale=alt.Scale(zero=False)),
        y=alt.Y("trestbps(systolic)", title="Systolic resting blood pressure (mm Hg)", scale=alt.Scale(zero=False)),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence")
    )
).configure_axis(titleFontSize=12)
chol_vs_restbps

alt.Chart(...)

In [389]:
chol_vs_max_htrt = (
    alt.Chart(hd_general)
    .mark_circle()
    .encode(
        x=alt.X("chol", title="Serum cholesterol level (mg/dl)", scale=alt.Scale(zero=False)),
        y=alt.Y("max_heart_rate", title="Maximum heart rate (BPM)", scale=alt.Scale(zero=False)),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence")
    )
).configure_axis(titleFontSize=12)
chol_vs_max_htrt

alt.Chart(...)

In [390]:
restbps_vs_max_htrt = (
    alt.Chart(hd_general)
    .mark_circle()
    .encode(
        x=alt.X("trestbps(systolic)", title="Systolic resting blood pressure (mm Hg)", scale=alt.Scale(zero=False)),
        y=alt.Y("max_heart_rate", title="Maximum heart rate (BPM)", scale=alt.Scale(zero=False)),
        color=alt.Color("heart_disease_presence", title="Heart Disease Presence")
    )
).configure_axis(titleFontSize=12)
restbps_vs_max_htrt

alt.Chart(...)